In [1]:
import random
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**Arena Steps**
- Create arena with investment opportunities (permanent).

In [2]:
class Arena:
    def __init__(self, size=10, mean=0.05, std=0.1, crowding_penalty=0.1):
        self.size = size
        self.mean = mean
        self.std = std
        self.crowding_penalty = crowding_penalty

        self.grid_columns = ["mean", "std"] # starting values, they do not change.
        self.grid = [[[mean, std] for _ in range(size)] for _ in range(size)]



    def get_flat_grid(self):
        # make each cell into dict with the self.grid_columns as keys
        flat_grid = []
        for i in range(self.size):
            for j in range(self.size):
                cell = {self.grid_columns[k]: self.grid[i][j][k] for k in range(len(self.grid_columns))}
                
                # Add x and y
                cell["x"] = i
                cell["y"] = j

                flat_grid.append(cell)
        return flat_grid
    

    def display(self):
        df = pd.DataFrame(self.grid)
        print(df)

arena = Arena(size=4)
arena.display()

             0            1            2            3
0  [0.05, 0.1]  [0.05, 0.1]  [0.05, 0.1]  [0.05, 0.1]
1  [0.05, 0.1]  [0.05, 0.1]  [0.05, 0.1]  [0.05, 0.1]
2  [0.05, 0.1]  [0.05, 0.1]  [0.05, 0.1]  [0.05, 0.1]
3  [0.05, 0.1]  [0.05, 0.1]  [0.05, 0.1]  [0.05, 0.1]


**Simulation Steps**
- Create simulation with $n$ agents and $m·m$ sized arena of opportunities.
- 

In [3]:
class Simulation:
    def __init__(self, num_agents=100, cycles=100, grid_size=5, types=None, history_window=3):
        self.num_agents = num_agents
        self.cycles = cycles
        self.cycle_idx = 0
        self.grid_size = grid_size
        self.types = types
        self.history_window = history_window

        # If types are provided, use them to create the initial agents.
        # types is expected to be a list of lists, e.g. [['A', 10], ['B', 5]]
        if self.types is not None:
            self.num_agents = sum([count for _, count in self.types])
        else:
            # If not provided, default to one type 'A' for all agents.
            self.types = [['A', num_agents]]
            self.num_agents = num_agents
        

        # Create initial agents with an additional 'type' column.
        agents_list = []
        for agent_type, count in self.types:
            for _ in range(count):
                agents_list.append({
                    'x': np.nan,
                    'y': np.nan,
                    'wealth': 100,
                    'status': 'alive',
                    'type': agent_type
                })
                agent_id += 1
        
        self.agents = pd.DataFrame(agents_list)
        # shuffle and reindex
        self.agents = self.agents.sample(frac=1).reset_index(drop=True)
        self.agents["id"] = range(len(self.agents))

        # Summary df
        type_columns = [type_name for type_name, _ in self.types]
        self.sim_summary = pd.DataFrame(columns=['cycle', 'agents', 'new_agents'] + type_columns, index=range(cycles))


    
    ###################
    # Helper functions #
    ###################
